In [1]:
from google.colab import files

upload = files.upload()

Saving teknofest_train.csv to teknofest_train.csv


In [7]:
import numpy as np
import pandas as pd
import re
import nltk
import ktrain
import string
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
nltk.download("stopwords")

stop_word_list = stopwords.words("turkish")

def preprocess_text(text):
  text = text.lower()
  text = re.sub(r"\n", " ", text)
  text = re.sub(r"\d", "", text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  words = text.split()
  words = [word for word in words if not word in stop_word_list]
  words = [re.sub(r"(.)\1{1,}", r"\1\1", word) for word in words]
  words = [word.strip() for word in words if len(word.strip()) > 1]
  text = " ".join(words)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("teknofest_train.csv", sep="|")

In [6]:
df["clean_text"] = df["text"].apply(preprocess_text)

In [9]:
X = df["clean_text"]
y = df["target"]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = np.array(X_test)
y_test = np.array(y_test)

In [13]:
classes = np.array(["INSULT", "RACIST", "SEXIST", "PROFANITY", "OTHER"])

In [14]:
from ktrain.text import Transformer

transformer = Transformer("loodos/albert-base-turkish-uncased", maxlen=400, classes=classes)

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


In [15]:
train = transformer.preprocess_train(X_train, y_train)
valid = transformer.preprocess_test(X_test, y_test)

preprocessing train...
language: tr
train sequence lengths:
	mean : 5
	95percentile : 12
	99percentile : 17


/usr/local/lib/python3.10/dist-packages/ktrain/utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


Is Multi-Label? False
preprocessing test...
language: tr
test sequence lengths:
	mean : 5
	95percentile : 12
	99percentile : 16


In [16]:
model = transformer.get_classifier()

In [19]:
learner = ktrain.get_learner(model, train_data=train, val_data=valid, batch_size=6)

In [20]:
learner.fit_onecycle(1e-6, 1)



begin training using onecycle policy with max lr of 1e-06...
1052/1052 [==============================] - 558s 529ms/step - loss: 1.3113 - accuracy: 0.5010 - val_loss: 0.9512 - val_accuracy: 0.7255


In [21]:
predictor = ktrain.get_predictor(learner.model, preproc=transformer)

In [22]:
predictor.predict("Sen yalanci sahtekar serefsiz bir adamsin.")

'INSULT'